In [1]:
"""
Purpose: Will test the whole process of restitching an entire mesh together


"""

'\nPurpose: Will test the whole process of restitching an entire mesh together\n\n\n'

In [3]:
import trimesh
import numpy as np
from collections import Counter
import time
import math
from tqdm import tqdm

# Whole Run that filters the mesh for only outside bodies

In [7]:
def filter_mesh_significant_outside_pieces(unfiltered_mesh,significance_threshold=2000,n_sample_points=1000):
    """
    Purpose; will take in a full, unfiltered mesh and find the biggest mesh piece, and then return a list of that mesh 
    with all of the other mesh fragments that are both above the significance_threshold AND outside of the biggest mesh piece

    Pseudocode: 
    1) split the meshes to unconnected pieces
    2) Filter the meshes for only those above the significance_threshold
    3) find the biggest mesh piece
    4) Iterate through all of the remaining pieces:
        a. Determine if mesh inside or outside main mesh
        b. If outside add to final list to return

    Returns: 
    1) list of significant mesh pieces, including the main one that are not inside of main mesh

    """

    mesh_pieces = unfiltered_mesh.split(only_watertight=False)
    
    print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []

    #find piece with largest size
    max_index = 0
    max_face_len = len(significant_pieces[max_index].faces)

    for i in range(1,len(significant_pieces)):
        if max_face_len < len(significant_pieces[i].faces):
            max_index = i
            max_face_len = len(significant_pieces[i].faces)

    print("max_index = " + str(max_index))
    print("max_face_len = " + str(max_face_len))

    final_mesh_pieces = []

    main_mesh = significant_pieces[max_index]

    #final_mesh_pieces.append(main_mesh)
    for i,mesh in enumerate(significant_pieces):
        if i != max_index:
            #get a random sample of points
            # points = np.array(mesh.vertices[:n_sample_points,:]) # OLD WAY OF DOING THIS
            idx = np.random.randint(len(mesh.vertices), size=n_sample_points)
            points = mesh.vertices[idx,:]
            
            
            start_time = time.time()
            signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
            print(f"Total time = {time.time() - start_time}")

            outside_percentage = sum(signed_distance < 0)/n_sample_points
            if outside_percentage > 0.9:
                final_mesh_pieces.append(mesh)
                print(f"Mesh piece {i} OUTSIDE mesh")
            else:
                print(f"Mesh piece {i} inside mesh :( ")
                
    return main_mesh,final_mesh_pieces

file_location = "./test_meshes/"
file_name ="example_neuron_gap_2.off"
unfiltered_mesh = trimesh.load_mesh(file_location + file_name)

#setting thresholds
significance_threshold=1 #number of faces needed for pieces to be considered to be kept
n_sample_points = 5 #number of points sampled on the mesh for determination of inside or outside
start_time = time.time()

#the main mesh is the first mesh in the piece
main_mesh,child_meshes = filter_mesh_significant_outside_pieces(unfiltered_mesh,
                            significance_threshold=significance_threshold,
                                n_sample_points=n_sample_points)
print(f"Total time for Mesh Cleansing: {time.time() - start_time}")


There were 4 pieces after mesh split
There were 4 pieces found after size threshold
max_index = 1
max_face_len = 12664
Total time = 0.1962568759918213
Mesh piece 0 OUTSIDE mesh
Total time = 0.004252433776855469
Mesh piece 2 OUTSIDE mesh
Total time = 0.012535333633422852
Mesh piece 3 OUTSIDE mesh
Total time for Mesh Cleansing: 0.3455660343170166


# Calculate the totally filtered facets and their center points

In [37]:
# need a way of finding the neighbors that have to share adjacent faces
def create_neighbors_lookup(mesh):
    start_time = time.time()
    neighbors_lookup = dict([(i,[]) for i in range(0,len(mesh.faces))])
    print(f"Creating empty dictionary : {time.time() - start_time}")
    
    start_time = time.time()
    for adj in mesh.face_adjacency:
        neighbors_lookup[adj[0]].append(adj[1])
        neighbors_lookup[adj[1]].append(adj[0])
    print(f"Filling in neighbors lookup : {time.time() - start_time}")
    
    return neighbors_lookup

def filter_and_expand_facets(new_mesh,
                             first_pass_size_threshold=2000,
                             normal_closeness=0.985):
    
    
    #filter the facets that are below a size threshold
    new_facets = new_mesh.facets[np.where(new_mesh.facets_area > first_pass_size_threshold)[0]]

    neighbors_lookup = create_neighbors_lookup(new_mesh)
    
    #generate a normals lookup table and see if works faster than regular lookup
    normal_lookup = {}

    for i in range(0,len(new_mesh.faces)):
        normal_lookup[i] = new_mesh.face_normals[i]
    
    global_start_time = time.time()
    final_facets= [0]*len(new_facets)
    
    for i,facet in enumerate(new_facets):
    #     mini_global_start_time = time.time()
    #     print(facet)
    #     print(type(facet))
    #     start_time = time.time()
    #     print([find_neighbors(k) for k in facet])
        total_neighbors = list(set(np.hstack([neighbors_lookup[k] for k in facet])).difference(set(facet)))
    #     print(total_neighbors)
    #     print(f"time initial neighbors: {(time.time() - start_time)}")
        neighbors_to_add = []
        neighbors_checked = []
        #print(total_neighbors)

        #just get the normal from one of the faces already in the facet
    #     start_time = time.time()
        facet_normal = normal_lookup[facet[0]]

    #     total_dot_time = 0
        while len(total_neighbors) > 0:
            current_neighbor = total_neighbors.pop()
            neighbors_checked.append(current_neighbor)

    #         print("--------------")
    # #         #check to see if neighbor has same normal face
    # #         start_dot_time = time.time()
    # #         dot_result = np.dot(new_mesh.face_normals[current_neighbor],facet_normal) #> normal_closeness
    # #         print(dot_result)
    # #         print((time.time() - start_dot_time) * 1000)

    #         start_dot_time = time.time()
    #         print(current_neighbor)
    #         #a = new_mesh.face_normals[current_neighbor]

    #         print((time.time() - start_dot_time) * 1000)
    #         dot_result = a[0]*facet_normal[0] + a[1]*facet_normal[1] + a[2]*facet_normal[2]  > normal_closeness
    #         print(dot_result)
    #         print((time.time() - start_dot_time) * 1000)
    #         print("--------------")

    #         total_dot_time += (time.time() - start_dot_time)*1000

            a = normal_lookup[current_neighbor]
            if a[0]*facet_normal[0] + a[1]*facet_normal[1] + a[2]*facet_normal[2]  > normal_closeness:

                neighbors_to_add.append(current_neighbor)
                #get the neighbors of this current face
                for neigh in neighbors_lookup[current_neighbor]:
                    #only add those neighbors that havent already been checked, in original facet group, or already in list to check
                    if neigh not in neighbors_checked and neigh not in facet and neigh not in total_neighbors:
                        total_neighbors.append(neigh)
    #     print(f"Total dot time: {total_dot_time}")
    #     print(f"time loop: {(time.time() - start_time)}")
    #     print("neighbors_to_add = " + str(neighbors_to_add))abs
    #     print("neighbors_checked = " + str(neighbors_checked))
    #     print("adding list = " + str(list(facet) + neighbors_to_add))
    #     start_time = time.time()
        final_facets[i] = list(facet) + neighbors_to_add
    #     print(f"Appending to list: {(time.time() - start_time)}")

    #     print(f"Total time: {(time.time() - mini_global_start_time)}")
    #     print("------------------------------------------------------")


    print(f"Total Facet building time: {(time.time() - global_start_time)}")
    return final_facets

def filter_final_facets(gap_mesh):
    
    final_facets = filter_and_expand_facets(gap_mesh,
                                 first_pass_size_threshold=2000,
                                 normal_closeness=0.985)

    # after computing final faces, filter for convexity
    edges = gap_mesh.edges_sorted.reshape((-1, 6)) #groups all the edges belonging to the corresponding face in one row
    final_facets_mean = np.zeros(len(final_facets))
    
    
    #make lookup table for face number to spot in the adjacency edges
    face_adjacency_index_lookup = [[] for i in gap_mesh.faces]
    for i,faces in enumerate(gap_mesh.face_adjacency):
        for f in faces:
            face_adjacency_index_lookup[f].append(i)


    for j,facet in tqdm(enumerate(final_facets)):
        # get the edges for each facet
        edges_facet = [edges[i].reshape((-1, 2)) for i in [facet]][0] #stores all the edges belonging to that face

        #get the indexes of the boundary edges:
        indexes = trimesh.grouping.group_rows(edges_facet, require_count=1)
        edge_0 = edges_facet[indexes]

        #find the faces that correspond to the boundary edges
        edge_0_faces = [facet[int(k/3)] for k in indexes]


        #2) Find the indexes of the edges int he face_adajacency_edges and store the projections
        adjacency_values = []
        for edge,edge_face in zip(edge_0,edge_0_faces):
            possible_adj_indexes = face_adjacency_index_lookup[edge_face]

            for index in possible_adj_indexes:
                if len(set(edge).intersection(set(gap_mesh.face_adjacency_edges[index]))) >= 2:
                    #print(f"adj edge = {e} and boundary edge = {edge}")
                    adjacency_values.append(gap_mesh.face_adjacency_angles[index]) # the metric we actually want to measure
                    break


        final_facets_mean[j] = np.mean(adjacency_values)
        
        

    #filter the final facets and output them so they can be plotted
    adjacency_threshold =  0.8
    final_facets_mean_filtered = np.array(final_facets)[final_facets_mean > adjacency_threshold]

    #Compute the centers
    final_facets_centers = []
    
    for filt in final_facets_mean_filtered: 
        #print("filt = " + str(filt))
        unique_vertices = gap_mesh.vertices[np.unique(gap_mesh.faces[filt].ravel())].astype("float")
        final_facets_centers.append((np.mean(unique_vertices[:,0]),
                          np.mean(unique_vertices[:,1]),
                          np.mean(unique_vertices[:,2])))
    
    return final_facets_mean_filtered,final_facets_centers


In [38]:
"""
Will calculate the facets for the main mesh and the children meshes


"""
main_mesh_facets,main_mesh_facets_centers = filter_final_facets(main_mesh)
child_meshes_facets = [filter_final_facets(gap_mesh) for gap_mesh in child_meshes]


Creating empty dictionary : 0.06480622291564941
Filling in neighbors lookup : 0.05715680122375488
Total Facet building time: 0.053648948669433594


368it [00:00, 2775.15it/s]
151it [00:00, 2751.01it/s]


Creating empty dictionary : 0.0012242794036865234
Filling in neighbors lookup : 0.010123252868652344
Total Facet building time: 0.03499317169189453
Creating empty dictionary : 0.002309560775756836
Filling in neighbors lookup : 0.018435955047607422


270it [00:00, 2970.67it/s]
31it [00:00, 2657.23it/s]

Total Facet building time: 0.046126365661621094
Creating empty dictionary : 9.322166442871094e-05
Filling in neighbors lookup : 0.0012698173522949219
Total Facet building time: 0.0034012794494628906


# Check that facet extraction for each piece worked

In [45]:
#len(child_meshes_facets[0][0])
#child_meshes_facets[0][1]

In [49]:
# download the facets list and see if the process worked:
index = 2
facets_group = np.zeros(len(child_meshes[index].faces)).astype(int)

for i,facet_group in enumerate(child_meshes_facets[index][0]):
    for face in facet_group:
        facets_group[face] = i + 1 #so that you reserve the label 0 for blenders none

np.savez("./test_meshes/" + file_name[:-4] + "_facet_piece_" + str(index) + ".npz",
         facets_group=facets_group,
        facets_group_centers=child_meshes_facets[index][1])

from print_trimesh import print_trimesh
print_trimesh(child_meshes[index],"./test_meshes/piece_" + str(index) + ".off")


len(facets_group)

596

In [ ]:
"""
Conclusion: 
1) Mesh children and main mesh generation done correctly
2) Mesh children and main facets correctly generated
3) Midpoints of facets are computed correctly 
    but it can be skewed towards sides with a lot of vertices because its the average

"""

# START ITERATIVE PROCESS THAT CONNECTS THE MESHES

In [82]:
"""
Pseudocode on how to filter according to bounding box
1) Get the min and max of the bounding box

In order for it to be within the range:
Every axis needs to be either one side in, 
both sides in,
or pass through it completely

When it does not do it:
1) both edges are less than or greater than the threshold

- Same algorithm as the soma contact
"""

def apply_bbox_filter(child,min_bb_zone,max_bb_zone):
    #get the min and max of the bounding box for the mesh
    min_bb = np.array(child.bounding_box.vertices).min(0)
    max_bb = np.array(child.bounding_box.vertices).max(0)
    
    #print(min_bb,max_bb)
    #print(min_bb_zone,max_bb_zone)
    
    #if fails any of these checks then return false, else return True
    if min(min_bb[0],max_bb[0])>max_bb_zone[0]:
        print("returning x greater max")
        return False
    
    if max(min_bb[0],max_bb[0])<min_bb_zone[0]:
        print("returning x less min")
        return False
    
    if min(min_bb[1],max_bb[1])>max_bb_zone[1]:
        print("returning y greater max")
        return False
    
    if max(min_bb[1],max_bb[1])<min_bb_zone[1]:
        print("returning y less min")
        return False
        
    if min(min_bb[2],max_bb[2])>max_bb_zone[2]:
        print("returning z greater max")
        return False
    
    if max(min_bb[2],max_bb[2])<min_bb_zone[2]:
        print("returning z less mim")
        return False
    return True

In [174]:
example_area = area(child_meshes[0].vertices[child_meshes[0].faces[14]]) #how to use the area function
print(example_area)
find_polygon_area(child_meshes[0],[14])

2156.509985232216


2156.509985232216

In [170]:
"""
Finds the polygon area given the mesh and the number of faces

"""

import math
def area(vertices):
    
    side_a = np.linalg.norm(vertices[0]-vertices[1])
    side_b = np.linalg.norm(vertices[1]-vertices[2])
    side_c = np.linalg.norm(vertices[2]-vertices[0])
    s = 0.5 * ( side_a + side_b + side_c)
    return math.sqrt(s * (s - side_a) * (s - side_b) * (s - side_c))

def find_polygon_area(mesh,list_of_faces):
    return(sum([area(mesh.vertices[mesh.faces[r]]) for r in list_of_faces]))
    

In [108]:
main_mesh.face_normals

In [334]:
"""
Notes: don't even need to have facets that are off limits because they will be destroyed after the remeshing process


"""

#declare main mesh

main_mesh = main_mesh
bounding_box_threshold = 4000
stitch_distance_threshold = 1000
size_ratio_threshold = 0.90


#final_mesh = trimesh.Trimesh()

#get the main mesh facets normals
main_mesh_normals = [main_mesh.face_normals[fac[0]] for fac in main_mesh_facets]
hit_indexes_list= []

#dictionary to save the stitch points
child_meshes_stitch_facets = dict()
child_meshes_stitch_face_ratios = dict()
for i,child in enumerate(child_meshes):
    
    #initialize the stitch index
    #child_meshes_stitch_facets[i] = [-1,-1]
    
    #two highest points for the bounding box
    min_bb = np.array(main_mesh.bounding_box.vertices).min(0)
    max_bb = np.array(main_mesh.bounding_box.vertices).max(0)
    
    min_bb_zone = min_bb - bounding_box_threshold
    max_bb_zone = max_bb + bounding_box_threshold
    
    
    
    #then send mesh to function that decides if with
    pass_bbox_filter = apply_bbox_filter(child,min_bb_zone,max_bb_zone)
    
    if not pass_bbox_filter:
        print("skipped")
        continue
    
    #### used for validation to make sure that bounding box was correctly filtering
#     #add the following to main mesh just to visualize
#     final_mesh = final_mesh + child
#     print("just added")
    
    #
    """
    
    Do pairwise calculation of distances between centers of facets on child piece and main piece
        Filter the pairs for stitch distance threshold
        Filter the pairs for matching or opposite normals
        Filter for those that are reasonable in size matching (within 50% of each other)

    If there are still ties after this point then pitch the best matching size

    If None remaining then keep going (might loosen up the parameters later), BUT WITH GLOBAL CHECK SO DOESN’T KEEP ITERATING


    """
    
    child_facets,child_facets_centers = child_meshes_facets[i]

    
    main_mesh_facets,main_mesh_facets_centers
    
    facet_distances = np.zeros((len(child_facets_centers),len(main_mesh_facets_centers)))
    facet_normals = np.zeros((len(child_facets_centers),len(main_mesh_facets_centers)))
    
    start_time = time.time()
    #do the pairwise comparison
    for cc,child_center in enumerate(child_facets_centers):
        
        #get the normal of the child
        child_normal = child.face_normals[child_facets[cc][0]]
        for mm, main_center in enumerate(main_mesh_facets_centers):
            #get the euclidean distance
#             facet_distances[cc,mm] = math.sqrt((child_center[0] - main_center[0])**2 + 
#                                                  (child_center[1] - main_center[1])**2 + 
#                                                  (child_center[2] - main_center[2])**2 )

            
    
            facet_distances[cc,mm] = np.linalg.norm(np.array(child_center) - np.array(main_center))
            
            main_normal = main_mesh_normals[mm]
#             print("main_normal = " + str(main_normal))
#             print("child_normal = " + str(child_normal))
            #get whether the normals are line up (or possibly opposite)
            normals_dot = np.dot(child_normal,main_normal)
            #print(normals_dot)
            facet_normals[cc,mm] =  normals_dot < -0.95  #or normals_dot > 0.95
                
            
    print(f"Child Index {i}")
    print(f"Total pairwise : {time.time() - start_time}")
    
#     if i == 1:
#         4,12
#         print("4,12 facet distance = " + str(facet_distances[4,12]))
        
    #apply the filters
    #stitch_distance_booleans = facet_distances < stitch_distance_threshold
    
    #get the indexes that make it through 
    hit_indexes = np.where(np.logical_and(facet_distances < stitch_distance_threshold,facet_normals)== True) 
    
    possible_stitch_pairs = np.vstack(hit_indexes).T
    
    face_0_unique_facets = np.unique(hit_indexes[0])
    face_1_unique_facets = np.unique(hit_indexes[1])
    #get the sizes of all the unique ones
    
    ####need to fix this up********************************* to find the area of the child facets and the main facets
    if len(hit_indexes[0]) <= 0:
        print("There was no possible stitch found after stitch distance and face normal filters")
        continue
    
    if len(hit_indexes[0]) > 0:
        face_0_facet_sizes = dict([(u,find_polygon_area(child,child_facets[u])) for u in face_0_unique_facets])
        face_1_facet_sizes = dict([(u,find_polygon_area(main_mesh,main_mesh_facets[u])) for u in face_1_unique_facets])
        
#     print("possible_stitch_pairs = " + str(possible_stitch_pairs))
#     print("len(hit_indexes) = " + str(len(hit_indexes)))
#     print("hit_indexes[0].any() = " + str(hit_indexes[0].any()))
#     print("hit_indexes = " + str(hit_indexes))
#     print("hit_indexes[0] = " + str(hit_indexes[0]))
#     print("len(hit_indexes[0]) = " + str(len(hit_indexes[0])))
    
    #find the sizes of all of them
    face_pair_sizes = np.zeros(len(possible_stitch_pairs[:,0]))
    face_size_ratios = np.zeros(len(possible_stitch_pairs[:,0]))
    
    #print("possible_stitch_pairs = " + str(possible_stitch_pairs))
    for numba,pair in enumerate(possible_stitch_pairs):
        #print(pair)
        sizes = [face_0_facet_sizes[pair[0]],face_1_facet_sizes[pair[1]]]
        min_area = min(sizes)
        max_area = max(sizes)
        
        ratio = min_area/max_area
        
        if ratio >= size_ratio_threshold:
            face_pair_sizes[numba] = min_area + max_area
            face_size_ratios[numba] = size_ratio_threshold
        
        
    #check that made it past stitch ratio threshold
    
    #best possible stitch pair is just the maximum sized matching ones
    best_index = np.where(face_pair_sizes == max(face_pair_sizes))
    best_stitch_pair = possible_stitch_pairs[best_index][0]
    best_stitch_pair_size = face_pair_sizes[best_index][0]
    best_stitch_pair_size_ratio = face_size_ratios[best_index][0]
    
    print("best_stitch_pair_size = " + str(best_stitch_pair_size))
    
    child_meshes_stitch_facets[i] = [best_stitch_pair[0],best_stitch_pair[1]]
    child_meshes_stitch_face_ratios[i] = best_stitch_pair_size_ratio
    
# makes sure that no two child branches try to connect to the same main branch
from collections import Counter
mesh_stitch_counter = Counter(np.array([val for val in child_meshes_stitch_facets.values()])[:,1])

repeat_main_facets = [key for key,val in mesh_stitch_counter.items() if (key != -1 and val > 1)] #gets the main mesh facet with multiples
print("repeat_main_facets = " + str(repeat_main_facets))


if len(repeat_main_facets)>0:
    
    child_mesh_double_indexes = [key for key,val in child_meshes_stitch_facets.items() if val[1] in repeat_main_facets]
    print("child_mesh_double_indexes = " + str(child_mesh_double_indexes))

    
    #decide which one to ditch --> pick the best matching area:
    max_ratio = -1
    max_child = -1

    
    for child_index in child_mesh_double_indexes:
        current_ratio = child_meshes_stitch_face_ratios[child_index]

        if current_ratio > max_ratio:
            max_child = child_index
            max_ratio = current_ratio

    print(f"max_child = {max_child}, max_ratio = {max_ratio}")

    #remove the others from the stitch facets
    for double_index in child_mesh_double_indexes:
        if double_index != max_child:
            del child_meshes_stitch_facets[double_index]
    
    
print(child_meshes_stitch_facets)



returning y greater max
skipped
Child Index 1
Total pairwise : 0.12802386283874512
best_stitch_pair_size = 2160988.278787873
Child Index 2
Total pairwise : 0.03378748893737793
There was no possible stitch found after stitch distance and face normal filters
repeat_main_facets = []
{1: [26, 36]}


# Restitching

In [376]:
import trimesh
import numpy as np
from collections import Counter
import time
import math

#gets the projection of point p onto line a
def ClosestPointOnLine(a, b, p):
    ap = p-a
    ab = b-a
    #base_vector = ab
    result = np.dot(ap,ab)/np.dot(ab,ab) # * ab
    return result




#now have the mesh and the facet faces, can send to function
def stitch_mesh_piece_vp3(new_mesh,facet_1,facet_2,
                          delete_facets=False,
                         return_added_mesh = True):
    """
    Changed since last version: 
    1) parameter for deleting facets at end or not
    2) parameter for returning added mesh or not 
    3) Changed normals check to print statement and not exception
    
    
    """
    #how to find the normals of facet groups:
    facet_group_1_normal = new_mesh.face_normals[facet_1[0]]
    facet_group_2_normal = new_mesh.face_normals[facet_2[0]]


    #get the correct version of the normals:
    if np.dot(facet_group_1_normal,facet_group_2_normal) > 0.8:
        pass
    elif np.dot(facet_group_1_normal,facet_group_2_normal) < -0.8:
        print("opposite normals")
        facet_group_2_normal = facet_group_2_normal*-1
    else:
        print("Not correct normals")
        #raise Exception("Not correct normals")

    print(facet_group_1_normal,facet_group_2_normal)

    # make each row correspond to a single face
    edges = new_mesh.edges_sorted.reshape((-1, 6))
    # get the edges for each facet
    edges_facet = [edges[i].reshape((-1, 2)) for i in [facet_1,facet_2]]
    edges_boundary = np.array([i[trimesh.grouping.group_rows(i, require_count=1)]
                               for i in edges_facet])

    edge_0 = edges_boundary[0]
    edge_1 = edges_boundary[1]

    #gets the unique number of points
    edge_0_points = np.unique(np.hstack(edge_0))
    edge_1_points = np.unique(np.hstack(edge_1))

    """
    get the dot product of all the points
    """

    #get any 2 points on the triangle and make that the reference traingle
    edge_0_anchor_points = new_mesh.vertices[[edge_0_points[0],edge_0_points[1]]]

    #gets the starting index for the 1st facet
    max_index = edge_0_points[0]
    max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[max_index])

    for i in range(1,len(edge_0_points)):
        current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_0_points[i]])

        if current_magnitude > max_magnitude:
            max_index = i
            max_magnitude = current_magnitude

    edge_0_starting_point = edge_0_points[max_index]

    #gets the starting index for the 2nd facet
    max_index = edge_1_points[0]
    max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[max_index])

    for i in range(1,len(edge_1_points)):
        current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_1_points[i]])
        if current_magnitude > max_magnitude:
            max_index = i
            max_magnitude = current_magnitude

    edge_1_starting_point = edge_1_points[max_index]

    print(f"starting edge 1st facet = {edge_0_starting_point}, starting edge 2nd facet= {edge_1_starting_point}, ")
    print(new_mesh.vertices[edge_0_starting_point],new_mesh.vertices[edge_1_starting_point])

    """
    Need to order the points for restitching

    Pseudocode: 
    1) Get starting piont
    2) Find the two edges corresponding to that point
    3) Need to decide which direction to start....
    - go in direction that make the cross of the (1st and last) point in the same direction of the 
    normal of the first facet
    4) loop through and record the orders of the vertices as you traverse along the edges 
    until you arrive back at the start
    5) Error if:
        a. You arrive back at the start and haven't processed all the edges
        b. Processsed all the edges but haven't arrived back at start

    6) Repeat steps 1 through 5 for 2nd facet group
    """
    start_point_list = [edge_0_starting_point,edge_1_starting_point]
    edge_list = [edge_0,edge_1]
    edge_order_list = []


    for i,start_point in enumerate(start_point_list):
        print(f"start_point = {start_point}")
        edge_order = [start_point]
        processed_edges = []

        #find the matching edges
        starting_edges_indices = np.where(np.logical_or(edge_list[i][:,0] == start_point,edge_list[i][:,1] == start_point) == True)[0]
        print(starting_edges_indices)
        starting_edges = edge_list[i][starting_edges_indices]
        print(f"starting edges = {starting_edges}") #the list of the two possible edges

        if starting_edges.size < 4:
            raise Exception("Not enough edges for 1st facet start point")

        if starting_edges.size > 4:
            raise Exception("Too many edges for 1st facet start point") 

        #np.where(starting_edges[1,:] != start_point)[0][0]
        #gets the vectors that will be used for the cross product
        print("np.where(starting_edges[0,:] != start_point)[0][0] = " + str(np.where(starting_edges[0,:] != start_point)[0][0]))
        print("np.where(starting_edges[1,:] != start_point)[0][0] = " + str(np.where(starting_edges[1,:] != start_point)[0][0]))


        possible_starting_vector_1 = new_mesh.vertices[starting_edges[0,:][np.where(starting_edges[0,:] != start_point)[0][0]]] - new_mesh.vertices[start_point]
        possible_starting_vector_2 = new_mesh.vertices[starting_edges[1,:][np.where(starting_edges[1,:] != start_point)[0][0]]] - new_mesh.vertices[start_point]

        #find the cross product of the starting vectors
        starting_edges_cross = np.cross(possible_starting_vector_1,possible_starting_vector_2)

        #make sure that the order of the vectors goes so that the cross product is in line with the starting normal
        if np.dot(starting_edges_cross,facet_group_1_normal) > 0:
            #print("group 1 picked")
            processed_edges.append(starting_edges_indices[0])
            current_vertex = starting_edges[0][np.where(starting_edges[0,:] != start_point)[0][0]]
        else:
            #print("group 2 picked")
            processed_edges.append(starting_edges_indices[1])
            #print("np.where(starting_edges[1,:] != start_point) = " + str(np.where(starting_edges[1,:] != start_point)))
            current_vertex = starting_edges[1][np.where(starting_edges[1,:] != start_point)[0][0]]

        #print(f"current_vertex = {current_vertex}" )

        #now iterate through number of 
        for z in range(1,edge_list[i][:,0].size):
            if current_vertex == start_point:
                raise Exception("Start vertex reached before processed all of edges")

            #get the next edge
            counter = 0
            next_vertex = -1
            for j,edg in enumerate(edge_list[i]):
                #print("edg = " + str(edg))
                #print("processed_edges = " + str(processed_edges))
                if current_vertex in edg and j not in processed_edges:
                    current_edge_index = j
                    if edg[0] != current_vertex:
                        next_vertex = edg[0]
                    else:
                        next_vertex = edg[1]


                    counter += 1
                    if counter >= 2:
                        raise Exception(f"More than 2 edges possibilities for {current_vertex}")

            #make sure the next vertex was found
            if next_vertex <= -1:
                raise Exception(f"No next vertex was found for {current_vertex} ")

            #if found next vertex
            processed_edges.append(current_edge_index)
            edge_order.append(current_vertex)

            current_vertex = next_vertex


        edge_order_list.append(edge_order)
        print(f"edge_{i}_order = {edge_order}")

    lengths_of_boundaries = [len(x) for x in edge_order_list]
    bigger = lengths_of_boundaries.index(max(lengths_of_boundaries))
    smaller = 1-bigger

    print(f"index of bigger facets = {bigger}\nindex of smaller facets = {smaller}",)

    dividend = int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])
    remainder = lengths_of_boundaries[bigger] - int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])*lengths_of_boundaries[smaller]

    print(f"dividend = {dividend}, remainder = {remainder}")


    new_faces = []
    current_bigger = 0
    for i,current_smaller in enumerate(edge_order_list[smaller]):
        print("current_smaller =" + str(current_smaller))
        print("current_bigger=" + str(current_bigger))
        if i == 0:
            new_faces.append([current_smaller,edge_order_list[smaller][-1],edge_order_list[bigger][current_bigger]])
        else:
            new_faces.append([current_smaller,edge_order_list[smaller][i-1],edge_order_list[bigger][current_bigger]])

        for j in range(0,dividend + int(i<remainder)):
            if current_bigger > len(edge_order_list[bigger]):
                raise Exception("Somehow rapped around too much")

            if current_bigger >= len(edge_order_list[bigger])-1:
                next_bigger = 0
            else:
                next_bigger = current_bigger+1

            new_faces.append([current_smaller,edge_order_list[bigger][current_bigger],
                                edge_order_list[bigger][next_bigger]])

            current_bigger += 1

    print("new_faces = " + str(new_faces))

    stitch_mesh = trimesh.Trimesh()

    stitch_mesh.vertices = new_mesh.vertices
    stitch_mesh.faces = new_mesh.faces
    stitch_mesh.faces = np.vstack([stitch_mesh.faces, new_faces])



    if delete_facets == True:
        #now take away the original facet faces:
        total_faces = np.linspace(0,len(stitch_mesh.faces)-1,len(stitch_mesh.faces)).astype("int")
        facet_faces = np.hstack([facet_1 ,facet_2])
        faces_to_keep = set(total_faces).difference(set(facet_faces))
        faces_to_keep

        stitch_mesh = stitch_mesh.submesh([list(faces_to_keep)])[0]

    trimesh.repair.fix_inversion(stitch_mesh)
    trimesh.repair.fix_winding(stitch_mesh)
    trimesh.repair.fix_normals(stitch_mesh)

    if return_added_mesh == True:
        added_mesh = trimesh.Trimesh()
        added_mesh.vertices = new_mesh.vertices
        added_mesh.faces = new_faces
        trimesh.repair.fix_inversion(added_mesh)
        trimesh.repair.fix_winding(added_mesh)
        trimesh.repair.fix_normals(added_mesh)

        return stitch_mesh,added_mesh
    
    else:
        return stitch_mesh

In [386]:
"""
Pseudocode for stitching:
1) For each pair in the child_meshes_stitch_facets:
a. Get the child mesh for that pair
b. Get the list of faces for the child facet (from the facet number)
c. Get the list of faces for the main facet (from the main number)

d. Get the original number of faces and vertices in the main mesh
d2. Use the orignal number of faces and add to list of faces for child facet to offset them correctly
    - Save this number list in a dictionary (to use for later and creating the submesh)
e. Add the two meshes together to get big mesh
f. Send the two meshes and the two facet lists to the restitching function to get a main mesh that is stitched up
 - but send it to function that doesnt delete the original facet faces 
     (because this would remove meshes from original and screw up facet number)
g. reassign the main_mesh to this newly stitched up mesh
h. recompute the facets for the main mesh



"""
child_faces_to_remove = []
current_main_mesh = main_mesh.copy()

#current_main_mesh.show() #checked that could make a copy of the mesh

main_faces_to_remove = []
for child_key,pair in child_meshes_stitch_facets.items():
    current_child_mesh = child_meshes[child_key]
    current_child_facet_faces = child_meshes_facets[child_key][0][pair[0]]
    
    
    current_main_mesh_facet_faces = main_mesh_facets[pair[1]]
    
    #Get the original number of faces and vertices in the main mesh
    original_mesh_faces_len = len(current_main_mesh.faces)
    current_child_facet_faces_adjusted = [k + original_mesh_faces_len 
                                                      for k in current_child_facet_faces]
    
    #Save the faces number for deletion later
    child_faces_to_remove += current_child_facet_faces_adjusted
    main_faces_to_remove += current_main_mesh_facet_faces
    
    combined_mesh = main_mesh + current_child_mesh
    
    #how to stitch up the mesh
    stitch_mesh,added_mesh = stitch_mesh_piece_vp3(new_mesh=combined_mesh,
                                                   facet_1=current_main_mesh_facet_faces,
                                                   facet_2=current_child_facet_faces_adjusted,
                                                  delete_facets=False,
                                                  return_added_mesh=True)
    
#     #reassign the main mesh
#     current_main_mesh = stitch_mesh
        
stitch_mesh.show()


opposite normals
[-0.09544989  0.99543424  0.        ] [ 0.03722231  0.99930701 -0.        ]
starting edge 1st facet = 4811, starting edge 2nd facet= 9574, 
[ 425443.6875       11419.42480469 -170083.234375  ] [ 425632.71875     11544.3828125 -170136.34375  ]
start_point = 4811
[20 27]
starting edges = [[2799 4811]
 [4811 5843]]
np.where(starting_edges[0,:] != start_point)[0][0] = 0
np.where(starting_edges[1,:] != start_point)[0][0] = 1
edge_0_order = [4811, 2799, 6349, 5007, 3460, 3412, 4148, 4802, 72, 70, 68, 67, 66, 65, 64, 63, 73, 71, 2455, 2690, 2324, 4159, 2703, 5718, 1961, 2751, 5183, 771, 4501, 5843]
start_point = 9574
[13 18]
starting edges = [[8037 9574]
 [9574 9866]]
np.where(starting_edges[0,:] != start_point)[0][0] = 0
np.where(starting_edges[1,:] != start_point)[0][0] = 1
edge_1_order = [9574, 9866, 9301, 7426, 7820, 10153, 7321, 6588, 10048, 9381, 9882, 7409, 9028, 9661, 6587, 8027, 10190, 9333, 7911, 10086, 10267, 10125, 7403, 8348, 9459, 9859, 10172, 8751, 9361, 7411, 

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [385]:
print(current_child_facet_faces[0])
print(current_child_facet_faces_adjusted[0])
print(len(current_main_mesh.faces))
print(len(main_mesh.faces))
print(original_mesh_faces_len)

3841
10193
12664
12664
6352


In [381]:
#download combined mesh and facets to make sure doing the right ones
file_name = "combined_test_1.off"
print_trimesh(combined_mesh,"./test_meshes/"+ file_name)

#download the facets to show labels

# download the facets list and see if the process worked:
facets_group = np.zeros(len(combined_mesh.faces)).astype(int)

for i,facet_group in enumerate([current_main_mesh_facet_faces,current_child_facet_faces_adjusted]):
    for face in facet_group:
        facets_group[face] = i + 1 #so that you reserve the label 0 for blenders none

np.savez("./test_meshes/" + file_name[:-4] + "_facets.npz",facets_group=facets_group)



# Testing that added meshes actually just stack:

In [341]:

main_mesh_off = trimesh.load_mesh("./test_meshes/main_mesh.off")
piece_1_off = trimesh.load_mesh("./test_meshes/piece_1.off")

print((len(main_mesh_off.vertices),len(main_mesh_off.faces)))
print((len(piece_1_off.vertices),len(piece_1_off.faces)))
print("combined")
print((len(piece_1_off.vertices) + len(main_mesh_off.vertices),len(piece_1_off.faces) + len(main_mesh_off.faces)))


(6352, 12664)
(4047, 8086)
combined
(10399, 20750)


In [340]:
combined_mesh = main_mesh_off + piece_1_off
print((len(combined_mesh.vertices),len(combined_mesh.faces))) 
"""
So the number of vertices and faces match 
"""

(10399, 20750)


In [355]:
#test to see if the first face matches the offset face in the combined mesh
piece_1_vertices = piece_1_off.vertices[piece_1_off.faces[0:len(piece_1_off.faces)]]
#print(piece_1_vertices)
whole_body_vertices = combined_mesh.vertices[combined_mesh.faces[len(main_mesh_off.faces):(len(main_mesh_off.faces) 
                                                                 + len(piece_1_off.faces))]]
#print(whole_body_vertices)

for piece,whole in zip(piece_1_vertices,whole_body_vertices):
    #print(piece)
    #print(whole)
    if not np.array_equal(piece,whole):
        print("not equal")
    #break
print("done running")

done running


In [357]:
#how to add same number to entire list
y = [4,5,6,78,9]
[i + 10 for i in y]

[14, 15, 16, 88, 19]

# TESTIN THE FILTER THAT MAKES SURE NO CHILD CAN CONNECT TO THE SAME FACET ON THE MAIN MESH

In [325]:
child_meshes_stitch_facets

{1: [26, 36], 3: [43, 36], 4: [76, 36]}

In [323]:
child_meshes_stitch_facets[3] = [43,36]
child_meshes_stitch_face_ratios[3] = 0.95

child_meshes_stitch_facets[4] = [76,36]
child_meshes_stitch_face_ratios[4] = 0.945

In [324]:
print(child_meshes_stitch_facets)
print(child_meshes_stitch_face_ratios)

{1: [26, 36], 3: [43, 36], 4: [76, 36]}
{1: 0.9, 3: 0.95, 4: 0.945}


In [327]:
#Could then do a counter
from collections import Counter
mesh_stitch_counter = Counter(np.array([val for val in child_meshes_stitch_facets.values()])[:,1])
mesh_stitch_counter
repeat_main_facets = [key for key,val in mesh_stitch_counter.items() if (key != -1 and val > 1)] #gets the main mesh facet with multiples
print(repeat_main_facets)

child_mesh_double_indexes = [key for key,val in child_meshes_stitch_facets.items() if val[1] in repeat_main_facets]
child_mesh_double_indexes


#decide which one to ditch --> pick the best matching area:
max_ratio = -1
max_child = -1

for child_index in child_mesh_double_indexes:
    current_ratio = child_meshes_stitch_face_ratios[child_index]
    
    if current_ratio > max_ratio:
        max_child = child_index
        max_ratio = current_ratio
        
print(f"max_child = {max_child}, max_ratio = {max_ratio}")

#remove the others from the stitch facets
for double_index in child_mesh_double_indexes:
    if double_index != max_child:
        del child_meshes_stitch_facets[double_index]
    
    
print(child_meshes_stitch_facets)


#then begin the restitching all of the facets



[]
max_child = -1, max_ratio = -1
{3: [43, 36]}
[36]


In [290]:
x = np.array([], dtype="int64")
x.any()

False

In [282]:
face_pair_sizes

array([], dtype=float64)

# Testing if facets they found are correct

In [279]:
npz_save_list =dict()

for key,value in child_meshes_stitch_facets.items():
    if value[0] != -1:
        print(key)
        print(child_meshes_facets[key][0][value[0]])
        print(main_mesh_facets[value[1]])
        
        npz_save_list[str(key)] = [child_meshes_facets[key][0][value[0]],main_mesh_facets[value[1]]]

#npz_save_list
"""[4008, 3917]
[2794, 8616]"""

1
[3841, 3840, 3851, 3838, 3837, 3839, 3835, 3834, 985, 625, 597, 1922, 2766, 3956, 3957, 3958, 3959, 3960, 3961, 3963, 3964, 641, 2618, 2617, 2547, 2740, 2739, 2171, 7502, 7503, 7515, 5642, 7527, 1466, 2689, 829, 642, 3132, 3049, 3050, 3047, 3127, 2792, 2276, 2275, 2302, 1680, 2793, 3519, 3679, 206, 3115, 2127, 2669, 5643, 5626, 5624, 5669, 7501, 7499, 7581, 6429, 1410, 4533, 6415, 6414, 6413, 6381, 4486]
[4856, 4849, 4850, 4851, 1753, 1751, 4758, 5365, 4771, 4770, 4898, 3123, 5371, 5503, 2020, 1997, 718, 2023, 2024, 2025, 5386, 2208, 2209, 1776, 1754, 4781, 4780, 4875]
2
[458, 591, 502, 590, 592, 593, 459, 460, 479, 466, 556, 467, 500, 485, 562, 455, 453, 530]
[4799, 4963, 4819, 4798, 463, 464, 2200, 2201]


'[4008, 3917]\n[2794, 8616]'

In [280]:
#save them off as an npz file
npz_save_list

np.savez("checking_facets.npz",**npz_save_list)

In [262]:
#test that it saved correctly
facets_groups_saved = np.load("checking_facets.npz")
facets_groups_saved["1"]
for key,value in facets_groups_saved.items():
    print(key)
    print(value)
    

1
[[4008 3917]
 [2794 8616]]


In [256]:
#print mesh childs
for i,child in enumerate(child_meshes):
    print_trimesh(child,"./test_meshes/piece_" + str(i) + ".off")

In [270]:
#this face is not being put in a facet pair --> see if can figure out why not:
mystery_face = 2618

for j,child in enumerate(child_meshes_facets[1][0]):
    if mystery_face in child:
        print(j)

#try to see when facets are dropping off
for example when is 4,12 not being true

4
20
25
26


In [271]:
main_mesh_facets[12]

[1754,
 1776,
 4875,
 4781,
 4780,
 1753,
 1751,
 4758,
 5365,
 4771,
 4770,
 4898,
 3123,
 5371,
 5503,
 2020,
 1997,
 718,
 2023,
 2024,
 5386,
 2208,
 2209,
 4856,
 4849,
 4850,
 4851,
 4900]

# shows that there are some overlapping facets 

In [164]:


example_child_facets = child_meshes_facets[1][0]

facets_on_face_1 = hit_indexes_list[0][0]
facets_on_face_2 = hit_indexes_list[0][1]

match_number = 0
non_match_number = 0

for zz in np.unique(facets_on_face_1):
    for yy in np.unique(facets_on_face_1):
        if zz != yy:
            #if len(set(example_child_facets[zz]).difference(set(example_child_facets[yy]))) == 0:
            if len(set(example_child_facets[zz]).intersection(set(example_child_facets[yy]))) > 0:
                match_number = match_number + 1
                #print((zz,yy))
            else:
                non_match_number = non_match_number + 1

print(match_number)
print(non_match_number)

42
1680


In [165]:
len(child_meshes_facets[1][0][12]),len(child_meshes_facets[1][0][13]),len(child_meshes_facets[1][0][38])

(17, 16, 17)

In [160]:
"""
Conclusion shows there is overlap but not always a huge amount
COULD GET RID OF TO HELP WITH EFFICIENCY


"""

[1282,
 1283,
 1275,
 1274,
 6780,
 6769,
 6767,
 6614,
 7412,
 6008,
 5582,
 5280,
 5277,
 4366,
 7507,
 5232]

In [147]:
example_child_facets = child_meshes_facets[2][0]
print(set(example_child_facets[11]).difference(set(example_child_facets[12])))
print(set(example_child_facets[12]).difference(set(example_child_facets[11])))
print(set(example_child_facets[12]).intersection(set(example_child_facets[11])))

for zz in 

set()
set()
{453, 485, 455, 458, 459, 460, 556, 590, 591, 592, 593, 466, 467, 500, 562, 502, 530, 479}


In [136]:
"""
Total pairwise : 0.08271956443786621
Total pairwise : 0.024506092071533203
"""


#gets the size of the facets

#sum(facet_normals.ravel() > 0)/len(facet_normals.ravel())

(array([11, 11, 11, 11, 11, 12, 12, 12, 12, 12]),
 array([ 4, 12, 33, 34, 36,  4, 12, 33, 34, 36]))